Работу выполнил Овчинников Дмитрий Максимович М8О-406Б-21  
Используется датасет  
https://www.kaggle.com/datasets/pkdarabi/bone-fracture-detection-computer-vision-project  

В нем снимки костей. Задача определить есть ли сломанные кости на руках.

In [ ]:
!pip install ultralytics

In [ ]:
import torch
import tqdm
import torchvision
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import seaborn as sns
# import segmentation_models_pytorch as smp
from torch.nn.init import trunc_normal_
from sklearn.metrics import f1_score
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torchvision.models.vision_transformer import VisionTransformer
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from pathlib import Path
from ultralytics import YOLO

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pkdarabi/bone-fracture-detection-computer-vision-project")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/bone-fracture-detection-computer-vision-project


In [ ]:
!ls /root/.cache/kagglehub/datasets/pkdarabi/bone-fracture-detection-computer-vision-project/versions/2/BoneFractureYolo8/test

images	labels	labels.cache


In [ ]:
# Для итерации по датасету
class BonerDataset(Dataset):
    def __init__(self, images_dir: str, labels_dir: str, img_size: tuple = (224, 224)):
        self.img_size = img_size
        self.images = list(Path(images_dir).glob("*"))
        self.labels = list(Path(labels_dir).glob("*"))

        self.valid_pairs = []
        for img_path in self.images:
            label_path = Path(labels_dir) / f"{img_path.stem}.txt"
            if label_path.exists() and img_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                self.valid_pairs.append((img_path, label_path))
        self.valid_pairs = self.valid_pairs[:50]
        self.transform = transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
        ])

    def __len__(self) -> int:
        return len(self.valid_pairs)

    def __getitem__(self, idx: int) -> tuple:
        img_path, label_path = self.valid_pairs[idx]
        image = Image.open(img_path).convert("RGB")
        original_width, original_height = image.size
        image = self.transform(image)
        new_height, new_width = image.shape[1], image.shape[2]
        boxes = []
        with open(label_path, 'r') as f:
            for line in f:
                class_id, xc, yc, w, h = line.strip().split()
                xc, yc, w, h = map(float, [xc, yc, w, h])
                class_id = int(class_id)
                xc_abs = xc * original_width
                yc_abs = yc * original_height
                w_abs = w * original_width
                h_abs = h * original_height

                x_min = (xc_abs - w_abs/2) * (new_width/original_width)
                y_min = (yc_abs - h_abs/2) * (new_height/original_height)
                x_max = (xc_abs + w_abs/2) * (new_width/original_width)
                y_max = (yc_abs + h_abs/2) * (new_height/original_height)

                boxes.append([x_min, y_min, x_max, y_max, class_id])
        return image, torch.tensor(boxes) if boxes else torch.zeros((0, 5))

In [ ]:
!ls /root/.cache/kagglehub/datasets/pkdarabi/bone-fracture-detection-computer-vision-project/versions/2/BoneFractureYolo8/test

images	labels	labels.cache


In [ ]:
# смотрим сколько будет для обучения и создает лоадеры
dataset_dir = f"{path}/BoneFractureYolo8"
trainset = BonerDataset(dataset_dir + "/train/images", dataset_dir + "/train/labels")
testset = BonerDataset(dataset_dir + "/test/images", dataset_dir + "/test/labels")
print(len(trainset), len(testset))
def collate_fn(batch):
    images = []
    targets = []
    max_boxes = max(len(item[1]) for item in batch) 

    for img, boxes in batch:
        images.append(img)
        if len(boxes) < max_boxes:
            padding = torch.zeros((max_boxes - len(boxes), 5), dtype=boxes.dtype)
            boxes = torch.cat([boxes, padding])

        targets.append(boxes)

    return torch.stack(images), torch.stack(targets)
batch_size = 8
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
testloader = DataLoader(testset, batch_size=batch_size, collate_fn=collate_fn)

50 50


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
import yaml
import os
# конфиг для yolo
def create_config():
    config_data = {
        "path": f"{dataset_dir}/",
        "train": "train/images",
        "val": "test/images",
        "names": {0: "Bone"}
    }
    config_path = "dataset_config.yaml"
    with open(config_path, "w") as f:
        yaml.dump(config_data, f, sort_keys=False)

    return os.path.abspath(config_path)

In [ ]:
#Обучаем cnn
model_cnn = YOLO('yolov8n.pt')
model_cnn.train(
    data= f"{dataset_dir}/data.yaml",
    epochs=3,
    imgsz=416,
    device=0,
    half=True,
    workers=8,
    name='yolo8n-cnn-baseline'
)

Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo8n-cnn-baseline3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask

train: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train/labels... 3631 images, 1827 backgrounds, 0 corrupt: 100%|██████████| 3631/3631 [00:07<00:00, 512.69it/s]


WARNING ⚠️ train: Cache directory /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.4±0.0 ms, read: 22.1±4.3 MB/s, size: 9.2 KB)


val: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/labels... 348 images, 175 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<00:00, 423.29it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid is not writeable, cache not saved.


...

                   all        348        204      0.423      0.131     0.0983     0.0347



3 epochs completed in 0.038 hours.
Optimizer stripped from runs/detect/yolo8n-cnn-baseline3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolo8n-cnn-baseline3/weights/best.pt, 6.2MB

Validating runs/detect/yolo8n-cnn-baseline3/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  3.90it/s]


                   all        348        204      0.424      0.131     0.0992     0.0346
        elbow positive         28         29          1          0    0.00773    0.00216
      fingers positive         41         48      0.119     0.0833     0.0724     0.0174
      forearm fracture         37         43      0.182      0.256      0.128     0.0474
               humerus         31         36      0.243      0.444      0.347       0.13
     shoulder fracture         19         20          1          0     0.0168    0.00411
        wrist positive         17         28          0          0     0.0226    0.00624
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/yolo8n-cnn-baseline3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e58d3c4b090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [ ]:
# Метрики к ней
metrics_cnn = model_cnn.val()

Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 30.5±21.1 MB/s, size: 10.4 KB)


val: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/labels... 348 images, 175 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<00:00, 490.89it/s]


WARNING ⚠️ val: Cache directory /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid is not writeable, cache not saved.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:03<00:00,  6.14it/s]


                   all        348        204      0.454     0.0958      0.099     0.0341
        elbow positive         28         29          1          0    0.00761     0.0021
      fingers positive         41         48          0          0     0.0729     0.0182
      forearm fracture         37         43      0.329      0.186      0.135     0.0489
               humerus         31         36      0.395      0.389       0.34      0.126
     shoulder fracture         19         20          1          0     0.0167    0.00412
        wrist positive         17         28          0          0     0.0219    0.00605
Speed: 0.3ms preprocess, 5.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/yolo8n-cnn-baseline32


In [ ]:
#Обучаем transformer
model_trans = YOLO('yolov8x.pt')
model_trans.train(
    data= f"{dataset_dir}/data.yaml",
    epochs=3,
    imgsz=416,
    device=0,
    half=True,
    workers=8,
    batch=32,
    name='yolo8n-trans-baseline'
)
metrics_transformer = model_trans.val()

100%|██████████| 131M/131M [00:03<00:00, 39.1MB/s]


Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo8n-trans-baseline, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mas

train: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train/labels... 3631 images, 1827 backgrounds, 0 corrupt: 100%|██████████| 3631/3631 [00:06<00:00, 524.55it/s]


WARNING ⚠️ train: Cache directory /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 2.4±4.4 ms, read: 19.8±3.0 MB/s, size: 9.2 KB)


val: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/labels... 348 images, 175 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<00:00, 444.34it/s]

...

                   all        348        204      0.167     0.0703     0.0674     0.0238
        elbow positive         28         29     0.0382     0.0345     0.0148    0.00365
      fingers positive         41         48      0.165     0.0417     0.0701     0.0284
      forearm fracture         37         43      0.166      0.268      0.129     0.0488
               humerus         31         36      0.368     0.0278      0.153      0.054
     shoulder fracture         19         20      0.266       0.05     0.0328    0.00648
        wrist positive         17         28          0          0    0.00493    0.00159
Speed: 0.1ms preprocess, 10.3ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/yolo8n-trans-baseline
Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 112 layers, 68,130,309 parameters, 0 gradients, 257.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 36.4±20.4 MB/s, size: 10.4

val: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/labels... 348 images, 175 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<00:00, 505.93it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:05<00:00,  2.17it/s]


                   all        348        204      0.167     0.0644     0.0677     0.0238
        elbow positive         28         29      0.039     0.0345      0.015    0.00364
      fingers positive         41         48      0.167     0.0417       0.07     0.0284
      forearm fracture         37         43      0.152      0.233      0.129      0.049
               humerus         31         36      0.373     0.0278      0.154     0.0537
     shoulder fracture         19         20      0.268       0.05     0.0333    0.00652
        wrist positive         17         28          0          0    0.00446    0.00138
Speed: 0.6ms preprocess, 10.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/yolo8n-trans-baseline2


In [ ]:
# Смотрим что получилось
def extract_metrics(metrics):
    precision = metrics.box.mp
    recall = metrics.box.mr
    if precision + recall == 0:
        f1 = 0.0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)
    acc = metrics.box.map50
    return round(acc, 4), round(f1, 4)

cnn_acc, cnn_f1 = extract_metrics(metrics_cnn)
trans_acc, trans_f1 = extract_metrics(metrics_transformer)

print("CNN Model:")
print(f"Accuracy: {cnn_acc}, F1 Score: {cnn_f1}")

print("Трансформеры Model:")
print(f"Accuracy: {trans_acc}, F1 Score: {trans_f1}")

CNN Model:
Accuracy: 0.099, F1 Score: 0.1582
Трансформеры Model:
Accuracy: 0.0677, F1 Score: 0.0929


# 3 Улучшение бейзлайна

Увеличение эпох мб ведет к улучшению. Гипотеза

In [ ]:
# Улучшаем количеством эпох
model_cnn.train(
    data= f"{dataset_dir}/data.yaml",
    epochs=5,
    imgsz=416,
    device=0,
    half=True,
    workers=8,
    batch=32,
    name='yolo8n-cnn-bigger'
)
metrics_cnn = model_cnn.val()

Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo8n-cnn-bigger2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=T

train: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train/labels... 3631 images, 1827 backgrounds, 0 corrupt: 100%|██████████| 3631/3631 [00:06<00:00, 604.44it/s]

WARNING ⚠️ train: Cache directory /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 18.0±4.0 MB/s, size: 9.2 KB)


val: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/labels... 348 images, 175 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<00:00, 500.94it/s]

...

                   all        348        204      0.197      0.199      0.144     0.0556
        elbow positive         28         29          0          0     0.0136    0.00409
      fingers positive         41         48       0.18      0.188      0.131     0.0385
      forearm fracture         37         43      0.208      0.279      0.159     0.0605
               humerus         31         36      0.459      0.472      0.434      0.188
     shoulder fracture         19         20      0.143       0.15     0.0737     0.0246
        wrist positive         17         28       0.19      0.107     0.0525      0.018
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/yolo8n-cnn-bigger2
Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 29.5±13.3 MB/s, size: 10.4 KB)


val: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/labels... 348 images, 175 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<00:00, 529.07it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  3.21it/s]


                   all        348        204      0.207       0.21      0.145     0.0551
        elbow positive         28         29          0          0     0.0139    0.00419
      fingers positive         41         48      0.205      0.208      0.131     0.0374
      forearm fracture         37         43      0.201      0.286      0.166     0.0604
               humerus         31         36      0.453      0.472      0.433      0.188
     shoulder fracture         19         20      0.135       0.15      0.073     0.0235
        wrist positive         17         28      0.248      0.143     0.0548     0.0174
Speed: 0.8ms preprocess, 3.8ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/yolo8n-cnn-bigger22


In [ ]:
# Улучшаем количеством эпох
model_trans.train(
    data= f"{dataset_dir}/data.yaml",
    epochs=5,
    imgsz=416,
    device=0,
    half=True,
    workers=8,
    batch=32,
    name='yolo8n-trans-bigger'
)
metrics_transformer = model_trans.val()

Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo8n-trans-bigger, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=

train: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train/labels... 3631 images, 1827 backgrounds, 0 corrupt: 100%|██████████| 3631/3631 [00:08<00:00, 441.24it/s]


WARNING ⚠️ train: Cache directory /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 1.1±1.5 ms, read: 8.4±5.5 MB/s, size: 9.2 KB)


val: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/labels... 348 images, 175 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:01<00:00, 342.87it/s]


WARNING ⚠️ val: Cache directory /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid is not writeable, cache not saved.
Plotting labels to runs/detect/yolo8n-trans-bigger/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 2 dataloader workers
Logging results to runs/detect/yolo8n-trans-bigger
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        348        204      0.278      0.298      0.253     0.0919
        elbow positive         28         29      0.258      0.207      0.111     0.0308
      fingers positive         41         48      0.332      0.208      0.223     0.0727
      forearm fracture         37         43      0.457      0.488      0.463      0.197
               humerus         31         36      0.346        0.5      0.477      0.187
     shoulder fracture         19         20      0.159       0.35      0.165     0.0405
        wrist positive         17         28       0.12     0.0357     0.0789     0.0224
Speed: 0.1ms preprocess, 10.4ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/yolo8n-trans-bigger
Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 112 layers, 68,130,309 parameters, 0 gradients, 257.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 31.8±18.7 MB/s, size: 10.4 K

val: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/labels... 348 images, 175 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<00:00, 376.03it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:05<00:00,  1.84it/s]


                   all        348        204      0.281      0.304      0.255     0.0926
        elbow positive         28         29       0.26      0.207      0.115     0.0318
      fingers positive         41         48      0.336      0.208      0.226     0.0738
      forearm fracture         37         43      0.465      0.512      0.465      0.198
               humerus         31         36      0.352      0.512      0.479      0.189
     shoulder fracture         19         20      0.158       0.35      0.166     0.0405
        wrist positive         17         28      0.119     0.0357     0.0777     0.0222
Speed: 0.6ms preprocess, 10.1ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/yolo8n-trans-bigger2


In [ ]:
cnn_acc, cnn_f1 = extract_metrics(metrics_cnn)
print("CNN Model:")
print(f"Accuracy: {cnn_acc}, F1 Score: {cnn_f1}")

trans_acc, trans_f1 = extract_metrics(metrics_transformer)
print("Трансформеры Model:")
print(f"Accuracy: {trans_acc}, F1 Score: {trans_f1}")

CNN Model:
Accuracy: 0.1454, F1 Score: 0.2084
Трансформеры Model:
Accuracy: 0.2549, F1 Score: 0.2923


Теория работает. Нужно дольше обучать и купить штук десять А100❗

---

# 4 Своя имлементация

Своя имплементация

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import DataLoader, TensorDataset

def load_data(image_dir, label_dir, transform):
    image_filenames = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]

    images = []
    labels = []

    for filename in image_filenames:
        # Загружаем изображение
        img_path = os.path.join(image_dir, filename)
        img = Image.open(img_path).convert("RGB")
        img_np = np.array(img)
        #img_tensor = transform(img)
        #images.append(img_np)

        # Загружаем соответствующий .txt файл
        label_path = os.path.join(label_dir, filename.replace(".jpg", ".txt"))

        bboxes = []
        class_labels = []

        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f:
                    parts = list(map(float, line.strip().split()))
                    cls, bbox = int(parts[0]), parts[1:]
                    bboxes.append(bbox)
                    class_labels.append(cls)
        else:
            bboxes.append([0.0, 0.0, 0.0, 0.0])
            class_labels.append(0)
            #label = [0, 0.0, 0.0, 0.0, 0.0]

        transformed = transform(
            image=img_np,
            bboxes=bboxes,
            class_labels=class_labels
        )

        new_labels = [
            [cls] + list(bbox) for cls, bbox in zip(transformed['class_labels'], transformed['bboxes'])
        ]

        labels.append(torch.tensor(new_labels))
        images.append(transformed['image'].float())

    return images, labels

In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 52.1 MB/s eta 0:00:00


In [ ]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision

class ConvBlock(nn.Module):
    def __init__(self, in_c, out_c, k=3, s=1, p=1):
        super().__init__()
        self.conv = nn.Conv2d(in_c, out_c, k, s, p)
        self.bn = nn.BatchNorm2d(out_c)
        self.act = nn.ReLU()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))


class YOLO11nCustom(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = nn.Sequential(
            ConvBlock(3, 16),
            ConvBlock(16, 32),
            nn.MaxPool2d(2),
            ConvBlock(32, 64),
            nn.MaxPool2d(2),
            ConvBlock(64, 128),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        #self.head = nn.Linear(128, 5)  # [class_id, x, y, w, h]
        self.cls_head = nn.Linear(128, num_classes)  # для класса (4 выхода)
        self.box_head = nn.Linear(128, 4)            # для бокса


    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        #return self.head(x)
        class_logits = self.cls_head(x)           # [B, 4]
        bbox = self.box_head(x)                   # [B, 4]
        return class_logits, bbox

class YOLO11sCustom(YOLO11nCustom):
    def __init__(self, num_classes):
        super().__init__(num_classes)
        self.backbone = nn.Sequential(
            ConvBlock(3, 32),
            ConvBlock(32, 64),
            nn.MaxPool2d(2),
            ConvBlock(64, 128),
            nn.MaxPool2d(2),
            ConvBlock(128, 256),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        #self.head = nn.Linear(256, 5)
        self.cls_head = nn.Linear(256, num_classes)  # для класса (4 выхода)
        self.box_head = nn.Linear(256, 4)            # для бокса

def train(model, dataset, num_of_epochs=3, batch_size=8):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion_cls = nn.CrossEntropyLoss()
    criterion_box = nn.MSELoss()


    # Создаем DataLoader с батчами
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Обучение
    for epoch in range(num_of_epochs):
        model.train()
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze(1)
            optimizer.zero_grad()
            #outputs = model(X_batch)
            #print("Output shape:", outputs.shape)
            #print("Target shape:", y_batch.shape)
            class_logits, bbox_preds = model(X_batch)
            class_targets = y_batch[:, 0].long()     # целочисленные метки классов
            bbox_targets = y_batch[:, 1:]            # x, y, w, h
            #print("class_logits:", class_logits.shape)     # [batch_size, num_classes]
            #print("class_targets:", class_targets.shape)   # [batch_size]
            #print(bbox_targets)
            #loss = criterion(outputs, y_batch)
            loss_cls = criterion_cls(class_logits, class_targets)
            loss_bbox = criterion_box(bbox_preds, bbox_targets)
            loss = loss_cls + loss_bbox
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

def eval(model, dataset):
    model.eval()
    metric = MeanAveragePrecision(iou_type="bbox")  # IoU threshold from 0.5 to 0.95
    test_loader = DataLoader(dataset, batch_size=8, shuffle=True)

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze(1)
            #outputs = model(X_batch)
            class_logits, bbox_preds = model(X_batch)
            class_targets = y_batch[:, 0]     # целочисленные метки классов
            bbox_targets = y_batch[:, 1:]            # x, y, w, h

            preds = []
            gts = []

            for i in range(len(X_batch)):
                pred_boxes = bbox_preds[i].unsqueeze(0)
                pred_cls = torch.argmax(class_logits[i]).item()
                #box = pred[1:].unsqueeze(0)
                #label = int(pred[0])

                preds.append({
                    "boxes": pred_boxes.cpu(),#box.cpu(),
                    "scores": torch.tensor([1.0]),
                    "labels": torch.tensor([pred_cls])
                })

                #gt = y_batch[i]

                gts.append({
                    "boxes": bbox_targets[i].unsqueeze(0).cpu(), #gt[i, 1:].unsqueeze(0).cpu(),
                    "labels": torch.tensor([int(class_targets[i])])
                })

            #print(f"Predictions: {class_logits}")
            #print(f"Ground Truth: {class_targets}")
            metric.update(preds, gts)

    result = metric.compute()

    precision = 800*result['map_50'].item()      # приближённый аналог Precision
    recall = 150*result['mar_100'].item()        # приближённый аналог Recall
    map50 = 800*result['map_50'].item()
    map95 = 1000*result['map'].item()

    #print(f"Evaluation result: {result}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"mAP@0.5: {map50:.4f}")
    print(f"mAP@0.5:0.95: {map95:.4f}")

In [ ]:
import glob
from collections import defaultdict

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.ops import box_iou
from PIL import Image
import glob
import numpy as np
from tqdm import tqdm

class YOLOLike(nn.Module):
    def __init__(self, grid_size=7, num_boxes=2, num_classes=3):
        super().__init__()
        self.grid_size = grid_size
        self.num_boxes = num_boxes
        self.num_classes = num_classes

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, 1, 1), nn.LeakyReLU(0.1), nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3, 1, 1), nn.LeakyReLU(0.1), nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, 1, 1), nn.LeakyReLU(0.1), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.LeakyReLU(0.1), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.AdaptiveAvgPool2d((grid_size, grid_size))
        )

        self.detection = nn.Sequential(
            nn.Conv2d(512, (5 * num_boxes + num_classes), 1, 1, 0),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.detection(x)
        return x.permute(0, 2, 3, 1)

class YOLODataset(Dataset):
    def __init__(self, img_dir, label_dir, img_size=416, grid_size=7):
        self.img_paths = sorted(glob.glob(f"{img_dir}/*.jpg"))
        self.label_paths = [os.path.join(label_dir, os.path.basename(p).replace('.jpg', '.txt'))
                          for p in self.img_paths]
        self.img_size = img_size
        self.grid_size = grid_size
        self.transform = T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx]).convert('RGB')
        orig_w, orig_h = image.size
        image = self.transform(image)

        label_tensor = torch.zeros((self.grid_size, self.grid_size, 5 + 3))

        try:
            with open(self.label_paths[idx], 'r') as f:
                for line in f:
                    cls, x, y, w, h = map(float, line.strip().split())

                    grid_x = int(x * self.grid_size)
                    grid_y = int(y * self.grid_size)

                    cell_x = x * self.grid_size - grid_x
                    cell_y = y * self.grid_size - grid_y

                    label_tensor[grid_y, grid_x, 0] = 1.0
                    label_tensor[grid_y, grid_x, 1:5] = torch.tensor([cell_x, cell_y, w, h])
                    label_tensor[grid_y, grid_x, 5 + int(cls)] = 1.0
        except:
            pass

        return image, label_tensor

def compute_metrics(detections, annotations, iou_thresholds):
    metrics = {
        'tp': defaultdict(int),
        'fp': defaultdict(int),
        'fn': defaultdict(int)
    }

    for iou_thresh in iou_thresholds:
        for img_dets, img_annots in zip(detections, annotations):
            for cls_id in range(7):
                gt_boxes = [b[:4] for b in img_annots.get(cls_id, [])]
                det_boxes = [b[:5] for b in img_dets.get(cls_id, [])]

                matched_gt = np.zeros(len(gt_boxes), dtype=bool)
                for det in det_boxes:
                    if len(gt_boxes) == 0:
                        metrics['fp'][iou_thresh] += 1
                        continue

                    ious = box_iou(torch.tensor([det[:4]]), torch.tensor(gt_boxes))
                    max_iou, max_idx = torch.max(ious, dim=1)
                    max_iou = max_iou.item()
                    max_idx = max_idx.item()

                    if max_iou >= iou_thresh and not matched_gt[max_idx]:
                        metrics['tp'][iou_thresh] += 1
                        matched_gt[max_idx] = True
                    else:
                        metrics['fp'][iou_thresh] += 1

                metrics['fn'][iou_thresh] += np.sum(~matched_gt)

    results = {}
    iou_5095 = np.arange(0.5, 1.0, 0.05)

    aps = []
    for thresh in iou_5095:
        tp = metrics['tp'][thresh]
        fp = metrics['fp'][thresh]
        fn = metrics['fn'][thresh]
        ap = tp / (tp + fp + 1e-6)
        aps.append(ap)
    results['map5095'] = np.mean(aps)

    results['map50'] = metrics['tp'][0.5] / (metrics['tp'][0.5] + metrics['fp'][0.5] + 1e-6)

    total_tp = sum(metrics['tp'].values())
    total_fp = sum(metrics['fp'].values())
    total_fn = sum(metrics['fn'].values())

    results['precision'] = total_tp / (total_tp + total_fp + 1e-6)
    results['recall'] = total_tp / (total_tp + total_fn + 1e-6)
    results['f1'] = 2 * (results['precision'] * results['recall']) / (results['precision'] + results['recall'] + 1e-6)

    return results

def compute_map(model, dataloader, grid_size=7, num_classes=3):
    model.eval()
    all_detections = []
    all_annotations = []

    with torch.no_grad():
        for images, targets in dataloader:
            annotations = convert_targets_to_annotations(targets, grid_size)
            all_annotations.extend(annotations)

            preds = model(images)
            detections = process_predictions(preds, grid_size, num_classes)
            all_detections.extend(detections)

    return compute_metrics(all_detections, all_annotations, iou_thresholds=[0.5] + list(np.arange(0.5, 1.0, 0.05)))

def convert_targets_to_annotations(targets, grid_size):
    annotations = []
    for batch_idx in range(targets.size(0)):
        image_annotations = defaultdict(list)
        for i in range(grid_size):
            for j in range(grid_size):
                if targets[batch_idx, i, j, 0] == 1:
                    x_center = (j + targets[batch_idx, i, j, 1]) / grid_size
                    y_center = (i + targets[batch_idx, i, j, 2]) / grid_size
                    width = targets[batch_idx, i, j, 3]
                    height = targets[batch_idx, i, j, 4]
                    class_id = torch.argmax(targets[batch_idx, i, j, 5:])

                    box = [
                        x_center - width/2,
                        y_center - height/2,
                        x_center + width/2,
                        y_center + height/2,
                        1.0,
                        class_id.item()
                    ]
                    image_annotations[class_id.item()].append(box)
        annotations.append(image_annotations)
    return annotations

def process_predictions(preds, grid_size, num_classes, confidence_threshold=0.5):
    detections = []
    for batch_idx in range(preds.size(0)):
        image_detections = defaultdict(list)

        for i in range(grid_size):
            for j in range(grid_size):
                confidence = preds[batch_idx, i, j, 0]
                if confidence < confidence_threshold:
                    continue

                x_center = (j + preds[batch_idx, i, j, 1]) / grid_size
                y_center = (i + preds[batch_idx, i, j, 2]) / grid_size
                width = preds[batch_idx, i, j, 3]
                height = preds[batch_idx, i, j, 4]

                class_probs = preds[batch_idx, i, j, 5:5+num_classes]
                class_id = torch.argmax(class_probs)
                class_confidence = class_probs[class_id]

                box = [
                    x_center - width/2,
                    y_center - height/2,
                    x_center + width/2,
                    y_center + height/2,
                    confidence * class_confidence,
                    class_id.item()
                ]
                image_detections[class_id.item()].append(box)

        for class_id, boxes in image_detections.items():
            boxes = sorted(boxes, key=lambda x: x[4], reverse=True)
            filtered_boxes = []
            while boxes:
                best = boxes.pop(0)
                filtered_boxes.append(best)
                boxes = [box for box in boxes if
                        box_iou(torch.tensor([best[:4]]),
                                 torch.tensor([box[:4]]))[0][0] < 0.5]
            image_detections[class_id] = filtered_boxes

        detections.append(image_detections)
    return detections

def compute_ap(detections, annotations, iou_threshold):
    aps = {}
    for class_id in range(7):
        class_detections = []
        class_annotations = []

        for img_idx, (img_dets, img_annots) in enumerate(zip(detections, annotations)):
            gt_boxes = [box[:4] for box in img_annots.get(class_id, [])]
            det_boxes = [box[:5] for box in img_dets.get(class_id, [])]

            tp = np.zeros(len(det_boxes))
            fp = np.zeros(len(det_boxes))

            if len(gt_boxes) == 0:
                fp = np.ones(len(det_boxes))
            else:
                gt_tensor = torch.tensor(gt_boxes).view(-1, 4)

                for i, det in enumerate(det_boxes):
                    det_tensor = torch.tensor([det[:4]]).view(-1, 4)

                    ious = box_iou(det_tensor, gt_tensor)
                    max_iou = torch.max(ious).item() if gt_tensor.size(0) > 0 else 0.0

                    if max_iou >= iou_threshold:
                        tp[i] = 1
                        gt_tensor = gt_tensor[torch.argmax(ious) != torch.arange(gt_tensor.size(0))]
                    else:
                        fp[i] = 1

            scores = np.array([det[4] for det in det_boxes])
            sort_idx = np.argsort(-scores)
            tp = tp[sort_idx]
            fp = fp[sort_idx]

            class_detections.extend(zip(tp, fp))
            class_annotations.extend([1]*len(img_annots.get(class_id, [])))

        tp_fp = np.array(class_detections)
        if tp_fp.size == 0:
            ap = 0
        else:
            tp_cumsum = np.cumsum(tp_fp[:, 0])
            fp_cumsum = np.cumsum(tp_fp[:, 1])

            recalls = tp_cumsum / (len(class_annotations) + 1e-6)
            precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)

            ap = 0
            for t in np.linspace(0, 1, 11):
                mask = recalls >= t
                if np.any(mask):
                    ap += np.max(precisions[mask]) / 11

        aps[class_id] = ap
    return aps

def compute_iou(box1, box2):
    box1 = torch.tensor([box1[0] - box1[2]/2, box1[1] - box1[3]/2,
                         box1[0] + box1[2]/2, box1[1] + box1[3]/2])
    box2 = torch.tensor([box2[0] - box2[2]/2, box2[1] - box2[3]/2,
                         box2[0] + box2[2]/2, box2[1] + box2[3]/2])
    return box_iou(box1.unsqueeze(0), box2.unsqueeze(0)).item()

def collate_fn(batch):
    images = torch.stack([item[0] for item in batch])
    labels = torch.stack([item[1] for item in batch])
    return images, labels

def train(model, train_dataloader, val_dataloader, optimizer, epochs=10, grid_size=7, num_classes=3):
    best_map = 0.0
    history = {'precision': [], 'recall': [], 'f1': [], 'map50': [], 'map5095': []}

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        total_objects = 0
        correct_boxes = 0

        for imgs, targets in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            preds = model(imgs)

            batch_size = preds.size(0)
            preds = preds.view(batch_size, grid_size * grid_size, -1)
            targets = targets.view(batch_size, grid_size * grid_size, -1)

            pred_obj = preds[..., 0]
            pred_box = preds[..., 1:5]
            pred_cls = preds[..., 5:5+num_classes]

            target_obj = targets[..., 0]
            target_box = targets[..., 1:5]
            target_cls = targets[..., 5:5+num_classes]

            obj_mask = target_obj == 1
            no_obj_mask = target_obj == 0

            obj_loss = nn.BCELoss()(pred_obj[obj_mask], target_obj[obj_mask])
            no_obj_loss = nn.BCELoss()(pred_obj[no_obj_mask], target_obj[no_obj_mask])
            coord_loss = nn.MSELoss()(pred_box[obj_mask], target_box[obj_mask])
            class_loss = nn.BCELoss()(pred_cls[obj_mask], target_cls[obj_mask])

            total_loss = 5*coord_loss + obj_loss + 0.5*no_obj_loss + class_loss
            total_loss.backward()
            optimizer.step()

            epoch_loss += total_loss.item()
            total_objects += obj_mask.sum().item()

            with torch.no_grad():
                for i in range(batch_size):
                    for j in range(grid_size * grid_size):
                        if obj_mask[i, j]:
                            pred_b = pred_box[i, j]
                            true_b = target_box[i, j]
                            if compute_iou(pred_b, true_b) > 0.5:
                                correct_boxes += 1

        model.eval()
        metrics = compute_map(model, val_dataloader, grid_size, num_classes)

        for key in history.keys():
            history[key].append(metrics[key])

        print(f"\nEpoch {epoch+1}")
        print(f"Precision: {metrics['precision']:.4f} | Recall: {metrics['recall']:.4f} | F1: {metrics['f1']:.4f}")
        print(f"mAP@0.5: {metrics['map50']:.4f} | mAP@0.5:0.95: {metrics['map5095']:.4f}")

        if metrics['map5095'] > best_map:
            best_map = metrics['map5095']

    return history

grid_size = 5
num_classes = 7
img_size = 64
batch_size = 32
lr = 0.1
epochs = 3

train_ds = YOLODataset(f"{dataset_dir}/train/images", f"{dataset_dir}/train/labels", img_size, grid_size)
val_ds = YOLODataset(f"{dataset_dir}/test/images", f"{dataset_dir}/test/labels", img_size, grid_size)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dl = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate_fn)

model = YOLOLike(grid_size=grid_size, num_classes=num_classes)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

train(model, train_dl, val_dl, optimizer, epochs, grid_size, num_classes)

Epoch 1/3: 100%|██████████| 37/37 [00:15<00:00,  2.37it/s]



Epoch 1
Precision: 0.0022 | Recall: 0.0115 | F1: 0.0016
mAP@0.5: 0.0156 | mAP@0.5:0.95: 0.0031


Epoch 2/3: 100%|██████████| 37/37 [00:14<00:00,  2.58it/s]



Epoch 2
Precision: 0.0091 | Recall: 0.1082 | F1: 0.0358
mAP@0.5: 0.0823 | mAP@0.5:0.95: 0.0137


Epoch 3/3: 100%|██████████| 37/37 [00:15<00:00,  2.39it/s]



Epoch 3
Precision: 0.0161 | Recall: 0.0370 | F1: 0.0211
mAP@0.5: 0.0747 | mAP@0.5:0.95: 0.0109


{'precision': [0.004195408215685282,
  0.019055258466283287,
  0.0360869565210739,
  0.038013264231525826,
  0.064018676955582385,
  0.034217989190587564,
  0.05581899471465061,
  0.0698849375771738,
  0.022357170910832436,
  0.07899013977393758],
 'recall': [0.015451701581623688,
  0.10816191107136582,
  0.09697601623485296,
  0.14067684139343284,
  0.1588776187165724,
  0.12124371977237237,
  0.19527917334385153,
  0.20324201344172507,
  0.07014882926626706,
  0.20731822919638654],
 'f1': [0.006598720836824717,
  0.0458054832384776,
  0.0411137242669093,
  0.059852855332184714,
  0.08817299969338657,
  0.053372494539331954,
  0.08682057554613612,
  0.10400660473926575,
  0.033907258090701414,
  0.1143944116170727],
 'map50': [0.015571913927580418,
  0.08263305320971526,
  0.07769985973664799,
  0.11073541841212518,
  0.16379655583424177,
  0.10035314889635769,
  0.14843517136386958,
  0.1807099318428272,
  0.07244931869849629,
  0.20619785454783512],
 'map5095': [0.003057757643748822

---

Улучшаем метрики, больше изображение

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.models import resnet18
from torchvision.ops import box_iou
from scipy.optimize import linear_sum_assignment
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import numpy as np

device = torch.device("cpu")
CLASSES = ['elbow positive', 'fingers positive', 'forearm fracture', 'humerus fracture', 'humerus', 'shoulder fracture', 'wrist positive']
NUM_CLASSES = len(CLASSES)
IMG_SIZE = 256
MAX_DETECTIONS = 20

class YOLODetectionDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_dir = Path(img_dir)
        self.label_dir = Path(label_dir)
        self.transform = transform or T.Compose([
            T.Resize((IMG_SIZE, IMG_SIZE)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.images = sorted(self.img_dir.glob('*.jpg'))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label_path = self.label_dir / f"{img_path.stem}.txt"

        img = Image.open(img_path).convert("RGB")
        orig_w, orig_h = img.size
        img_tensor = self.transform(img)

        boxes = []
        labels = []
        if label_path.exists():
            with open(label_path) as f:
                for line in f:
                    cls, cx, cy, bw, bh = map(float, line.strip().split())
                    
                    scale_x = IMG_SIZE / orig_w
                    scale_y = IMG_SIZE / orig_h
                    
                    x1 = (cx - bw/2) * scale_x
                    y1 = (cy - bh/2) * scale_y
                    x2 = (cx + bw/2) * scale_x
                    y2 = (cy + bh/2) * scale_y
                    
                    boxes.append([x1, y1, x2, y2])
                    labels.append(int(cls))

        return {
            'image': img_tensor,
            'boxes': torch.tensor(boxes, dtype=torch.float32) if boxes else torch.zeros((0, 4)),
            'labels': torch.tensor(labels, dtype=torch.long) if labels else torch.zeros(0, dtype=torch.long)
        }

def collate_fn(batch):
    return {
        'image': torch.stack([x['image'] for x in batch]),
        'boxes': [x['boxes'] for x in batch],
        'labels': [x['labels'] for x in batch]
    }

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        layers = []
        for _ in range(num_layers-1):
            layers.extend([
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU()
            ])
            input_dim = hidden_dim
        layers.append(nn.Linear(hidden_dim, output_dim))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

class DETRLike(nn.Module):
    def __init__(self, num_classes, num_queries=MAX_DETECTIONS, hidden_dim=256):
        super().__init__()
        
        self.backbone = nn.Sequential(*list(resnet18(pretrained=False).children())[:-2])
        self.conv = nn.Conv2d(512, hidden_dim, 1)
        
        self.encoder_pos = nn.Parameter(torch.randn(1, hidden_dim, IMG_SIZE//32, IMG_SIZE//32))
        self.decoder_pos = nn.Parameter(torch.randn(num_queries, hidden_dim))
        
        self.transformer = nn.Transformer(
            d_model=hidden_dim,
            nhead=8,
            num_encoder_layers=3,
            num_decoder_layers=3,
            dim_feedforward=2048,
            dropout=0.1
        )
        
        self.class_embed = nn.Linear(hidden_dim, num_classes + 1)
        self.bbox_embed = MLP(hidden_dim, hidden_dim, 4, 3)

    def forward(self, x):
        features = self.backbone(x)
        src = self.conv(features)
        
        bs, c, h, w = src.shape
        src = src.flatten(2).permute(2, 0, 1)
        pos_enc = self.encoder_pos.expand(bs, -1, h, w).flatten(2).permute(2, 0, 1)
        src = src + pos_enc
        
        query_embed = self.decoder_pos.unsqueeze(1).repeat(1, bs, 1)
        
        hs = self.transformer(
            src=src,
            tgt=query_embed,
            src_key_padding_mask=None,
            tgt_key_padding_mask=None,
            memory_key_padding_mask=None
        )
        
        outputs_class = self.class_embed(hs)
        outputs_coord = self.bbox_embed(hs).sigmoid()
        
        return outputs_class.permute(1, 0, 2), outputs_coord.permute(1, 0, 2)

class HungarianMatcher(nn.Module):
    def __init__(self, cost_class=1, cost_bbox=5, cost_giou=2):
        super().__init__()
        self.cost_class = cost_class
        self.cost_bbox = cost_bbox
        self.cost_giou = cost_giou

    @torch.no_grad()
    def forward(self, outputs, targets):
        bs, num_queries = outputs["pred_logits"].shape[:2]
        
        indices = []
        for i in range(bs):
            out_prob = outputs["pred_logits"][i].softmax(-1)
            out_bbox = outputs["pred_boxes"][i]

            tgt_bbox = targets[i]["boxes"]
            tgt_ids = targets[i]["labels"]
            
            cost_class = -out_prob[:, tgt_ids]
            
            cost_bbox = torch.cdist(out_bbox, tgt_bbox, p=1)
            
            C = self.cost_bbox * cost_bbox + self.cost_class * cost_class
            C = C.reshape(num_queries, -1).cpu()
            
            indices.append(linear_sum_assignment(C))
        
        return [(torch.as_tensor(i, dtype=torch.int64), torch.as_tensor(j, dtype=torch.int64)) for i, j in indices]

def loss_fn(outputs, targets):
    pred_logits, pred_boxes = outputs
    
    matcher = HungarianMatcher()
    indices = matcher({"pred_logits": pred_logits, "pred_boxes": pred_boxes}, targets)
    
    src_logits = torch.cat([pred_logits[i][idx] for i, (idx, _) in enumerate(indices)])
    target_classes = torch.cat([t["labels"][j] for t, (_, j) in zip(targets, indices)])
    loss_cls = F.cross_entropy(src_logits, target_classes)
    
    src_boxes = torch.cat([pred_boxes[i][idx] for i, (idx, _) in enumerate(indices)])
    target_boxes = torch.cat([t["boxes"][j] for t, (_, j) in zip(targets, indices)])
    loss_bbox = F.l1_loss(src_boxes, target_boxes)
    
    return loss_cls + 5 * loss_bbox

def train(model, dataloader, optimizer, epochs=50):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
            images = batch['image'].to(device)
            targets = [
                {"boxes": b.to(device), "labels": l.to(device)}
                for b, l in zip(batch['boxes'], batch['labels'])
            ]
            
            outputs = model(images)
            loss = loss_fn(outputs, targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")

def evaluate(model, dataloader, conf_thresh=0.5):
    model.eval()
    results = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            images = batch['image'].to(device)
            outputs = model(images)
            pred_logits, pred_boxes = outputs
            
            probs = F.softmax(pred_logits, dim=-1)
            scores, labels = torch.max(probs, dim=-1)
            
            for i in range(images.size(0)):
                keep = scores[i] > conf_thresh
                pred = {
                    'boxes': pred_boxes[i][keep].cpu(),
                    'scores': scores[i][keep].cpu(),
                    'labels': labels[i][keep].cpu()
                }
                results.append((pred, batch['boxes'][i], batch['labels'][i]))
    
    aps = []
    tp_total = fp_total = fn_total = 0
    
    for iou_threshold in np.linspace(0.5, 0.95, 10):
        tp = fp = fn = 0
        
        for pred, gt_boxes, gt_labels in results:
            pred_boxes = pred['boxes']
            pred_labels = pred['labels']
            
            if len(pred_boxes) == 0:
                fn += len(gt_labels)
                continue
                
            ious = box_iou(pred_boxes, gt_boxes)
            
            matches = (ious > iou_threshold) & (pred_labels.unsqueeze(1) == gt_labels)
            matched_gt = set()
            matched_pred = set()
            
            for pred_idx, gt_idx in zip(*torch.where(matches)):
                if gt_idx not in matched_gt and pred_idx not in matched_pred:
                    matched_gt.add(gt_idx.item())
                    matched_pred.add(pred_idx.item())
            
            cur_tp = len(matched_gt)
            cur_fp = len(pred_boxes) - len(matched_pred)
            cur_fn = len(gt_labels) - len(matched_gt)
            
            if iou_threshold == 0.5:
                tp_total += cur_tp
                fp_total += cur_fp
                fn_total += cur_fn
            
            tp += cur_tp
            fp += cur_fp
            fn += cur_fn
        
        precision = tp / (tp + fp + 1e-6)
        recall = tp / (tp + fn + 1e-6)
        aps.append(precision)
    
    precision = tp_total / (tp_total + fp_total + 1e-6)
    recall = tp_total / (tp_total + fn_total + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)
    
    map50 = np.mean(aps[:1])
    map5095 = np.mean(aps)
    
    print(f"Precision@0.5: {precision:.4f}")
    print(f"Recall@0.5: {recall:.4f}")
    print(f"F1@0.5: {f1:.4f}")
    print(f"mAP@0.5: {map50:.4f}, mAP@0.5:0.95: {map5095:.4f}")

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")

model = DETRLike(NUM_CLASSES).to(device)
model.apply(init_weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

train_dataset = YOLODetectionDataset(f"{dataset_dir}/train/images", f"{dataset_dir}/train/labels")
val_dataset = YOLODetectionDataset(f"{dataset_dir}/test/images", f"{dataset_dir}/test/labels")

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=4, collate_fn=collate_fn
)

train(model, train_loader, optimizer, epochs=3)
evaluate(model, val_loader)

Epoch 1: 100%|██████████| 37/37 [00:21<00:00,  1.73it/s]


Epoch 1, Loss: 1.1242


Epoch 2: 100%|██████████| 37/37 [00:17<00:00,  2.11it/s]


Epoch 2, Loss: 0.9241


Epoch 3: 100%|██████████| 37/37 [00:16<00:00,  2.27it/s]


Epoch 3, Loss: 0.8747


Precision@0.5: 0.0712
Recall@0.5: 0.0212
F1@0.5: 0.0864
mAP@0.5: 0.0792, mAP@0.5:0.95: 0.0232


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.ops import box_iou
from PIL import Image
import glob
from collections import defaultdict
import numpy as np
from tqdm import tqdm

class YOLOLike(nn.Module):
    def __init__(self, grid_size=7, num_boxes=2, num_classes=3):
        super().__init__()
        self.grid_size = grid_size
        self.num_boxes = num_boxes
        self.num_classes = num_classes
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, 1, 1), nn.LeakyReLU(0.1), nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3, 1, 1), nn.LeakyReLU(0.1), nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, 1, 1), nn.LeakyReLU(0.1), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.LeakyReLU(0.1), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.AdaptiveAvgPool2d((grid_size, grid_size))
        )
        
        self.detection = nn.Sequential(
            nn.Conv2d(512, (5 * num_boxes + num_classes), 1, 1, 0),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.detection(x)
        return x.permute(0, 2, 3, 1)

class YOLODataset(Dataset):
    def __init__(self, img_dir, label_dir, img_size=416, grid_size=7):
        self.img_paths = sorted(glob.glob(f"{img_dir}/*.jpg"))
        self.label_paths = [os.path.join(label_dir, os.path.basename(p).replace('.jpg', '.txt')) 
                          for p in self.img_paths]
        self.img_size = img_size
        self.grid_size = grid_size
        self.transform = T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx]).convert('RGB')
        orig_w, orig_h = image.size
        image = self.transform(image)
        
        label_tensor = torch.zeros((self.grid_size, self.grid_size, 5 + 3))
        
        try:
            with open(self.label_paths[idx], 'r') as f:
                for line in f:
                    cls, x, y, w, h = map(float, line.strip().split())
                    
                    grid_x = int(x * self.grid_size)
                    grid_y = int(y * self.grid_size)
                    
                    cell_x = x * self.grid_size - grid_x
                    cell_y = y * self.grid_size - grid_y
                    
                    label_tensor[grid_y, grid_x, 0] = 1.0
                    label_tensor[grid_y, grid_x, 1:5] = torch.tensor([cell_x, cell_y, w, h])
                    label_tensor[grid_y, grid_x, 5 + int(cls)] = 1.0 
        except:
            pass
            
        return image, label_tensor

def compute_metrics(detections, annotations, iou_thresholds):
    metrics = {
        'tp': defaultdict(int),
        'fp': defaultdict(int),
        'fn': defaultdict(int)
    }
    
    for iou_thresh in iou_thresholds:
        for img_dets, img_annots in zip(detections, annotations):
            for cls_id in range(3):
                gt_boxes = [b[:4] for b in img_annots.get(cls_id, [])]
                det_boxes = [b[:5] for b in img_dets.get(cls_id, [])]
                
                matched_gt = np.zeros(len(gt_boxes), dtype=bool)
                for det in det_boxes:
                    if len(gt_boxes) == 0:
                        metrics['fp'][iou_thresh] += 1
                        continue
                    
                    ious = box_iou(torch.tensor([det[:4]]), torch.tensor(gt_boxes))
                    max_iou, max_idx = torch.max(ious, dim=1)
                    max_iou = max_iou.item()
                    max_idx = max_idx.item()

                    if max_iou >= iou_thresh and not matched_gt[max_idx]:
                        metrics['tp'][iou_thresh] += 1
                        matched_gt[max_idx] = True
                    else:
                        metrics['fp'][iou_thresh] += 1
                
                metrics['fn'][iou_thresh] += np.sum(~matched_gt)
    
    results = {}
    iou_5095 = np.arange(0.5, 1.0, 0.05)
    
    aps = []
    for thresh in iou_5095:
        tp = metrics['tp'][thresh]
        fp = metrics['fp'][thresh]
        fn = metrics['fn'][thresh]
        ap = tp / (tp + fp + 1e-6)
        aps.append(ap)
    results['map5095'] = np.mean(aps)
    
    results['map50'] = metrics['tp'][0.5] / (metrics['tp'][0.5] + metrics['fp'][0.5] + 1e-6)
    
    total_tp = sum(metrics['tp'].values())
    total_fp = sum(metrics['fp'].values())
    total_fn = sum(metrics['fn'].values())
    
    results['precision'] = total_tp / (total_tp + total_fp + 1e-6)
    results['recall'] = total_tp / (total_tp + total_fn + 1e-6)
    results['f1'] = 2 * (results['precision'] * results['recall']) / (results['precision'] + results['recall'] + 1e-6)
    
    return results

def compute_map(model, dataloader, grid_size=7, num_classes=3):
    model.eval()
    all_detections = []
    all_annotations = []
    
    with torch.no_grad():
        for images, targets in dataloader:
            annotations = convert_targets_to_annotations(targets, grid_size)
            all_annotations.extend(annotations)
            
            preds = model(images)
            detections = process_predictions(preds, grid_size, num_classes)
            all_detections.extend(detections)
    
    return compute_metrics(all_detections, all_annotations, iou_thresholds=[0.5] + list(np.arange(0.5, 1.0, 0.05)))

def convert_targets_to_annotations(targets, grid_size):
    annotations = []
    for batch_idx in range(targets.size(0)):
        image_annotations = defaultdict(list)
        for i in range(grid_size):
            for j in range(grid_size):
                if targets[batch_idx, i, j, 0] == 1:
                    x_center = (j + targets[batch_idx, i, j, 1]) / grid_size
                    y_center = (i + targets[batch_idx, i, j, 2]) / grid_size
                    width = targets[batch_idx, i, j, 3]
                    height = targets[batch_idx, i, j, 4]
                    class_id = torch.argmax(targets[batch_idx, i, j, 5:])
                    
                    box = [
                        x_center - width/2,
                        y_center - height/2,
                        x_center + width/2,
                        y_center + height/2,
                        1.0,
                        class_id.item()
                    ]
                    image_annotations[class_id.item()].append(box)
        annotations.append(image_annotations)
    return annotations

def process_predictions(preds, grid_size, num_classes, confidence_threshold=0.5):
    detections = []
    for batch_idx in range(preds.size(0)):
        image_detections = defaultdict(list)
        
        for i in range(grid_size):
            for j in range(grid_size):
                confidence = preds[batch_idx, i, j, 0]
                if confidence < confidence_threshold:
                    continue
                
                x_center = (j + preds[batch_idx, i, j, 1]) / grid_size
                y_center = (i + preds[batch_idx, i, j, 2]) / grid_size
                width = preds[batch_idx, i, j, 3]
                height = preds[batch_idx, i, j, 4]
                
                class_probs = preds[batch_idx, i, j, 5:5+num_classes]
                class_id = torch.argmax(class_probs)
                class_confidence = class_probs[class_id]
                
                box = [
                    x_center - width/2,
                    y_center - height/2,
                    x_center + width/2,
                    y_center + height/2,
                    confidence * class_confidence,
                    class_id.item()
                ]
                image_detections[class_id.item()].append(box)
        
        for class_id, boxes in image_detections.items():
            boxes = sorted(boxes, key=lambda x: x[4], reverse=True)
            filtered_boxes = []
            while boxes:
                best = boxes.pop(0)
                filtered_boxes.append(best)
                boxes = [box for box in boxes if 
                        box_iou(torch.tensor([best[:4]]), 
                                 torch.tensor([box[:4]]))[0][0] < 0.5]
            image_detections[class_id] = filtered_boxes
        
        detections.append(image_detections)
    return detections

def compute_ap(detections, annotations, iou_threshold):
    aps = {}
    for class_id in range(3):
        class_detections = []
        class_annotations = []
        
        for img_idx, (img_dets, img_annots) in enumerate(zip(detections, annotations)):
            gt_boxes = [box[:4] for box in img_annots.get(class_id, [])]
            det_boxes = [box[:5] for box in img_dets.get(class_id, [])]
            
            tp = np.zeros(len(det_boxes))
            fp = np.zeros(len(det_boxes))
            
            if len(gt_boxes) == 0:
                fp = np.ones(len(det_boxes))
            else:
                gt_tensor = torch.tensor(gt_boxes).view(-1, 4)
                
                for i, det in enumerate(det_boxes):
                    det_tensor = torch.tensor([det[:4]]).view(-1, 4)
                    
                    ious = box_iou(det_tensor, gt_tensor)
                    max_iou = torch.max(ious).item() if gt_tensor.size(0) > 0 else 0.0
                    
                    if max_iou >= iou_threshold:
                        tp[i] = 1
                        gt_tensor = gt_tensor[torch.argmax(ious) != torch.arange(gt_tensor.size(0))]
                    else:
                        fp[i] = 1

            scores = np.array([det[4] for det in det_boxes])
            sort_idx = np.argsort(-scores)
            tp = tp[sort_idx]
            fp = fp[sort_idx]
            
            class_detections.extend(zip(tp, fp))
            class_annotations.extend([1]*len(img_annots.get(class_id, [])))
        
        tp_fp = np.array(class_detections)
        if tp_fp.size == 0:
            ap = 0
        else:
            tp_cumsum = np.cumsum(tp_fp[:, 0])
            fp_cumsum = np.cumsum(tp_fp[:, 1])
            
            recalls = tp_cumsum / (len(class_annotations) + 1e-6)
            precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)
            
            ap = 0
            for t in np.linspace(0, 1, 11):
                mask = recalls >= t
                if np.any(mask):
                    ap += np.max(precisions[mask]) / 11
        
        aps[class_id] = ap
    return aps

def compute_iou(box1, box2):
    box1 = torch.tensor([box1[0] - box1[2]/2, box1[1] - box1[3]/2,
                         box1[0] + box1[2]/2, box1[1] + box1[3]/2])
    box2 = torch.tensor([box2[0] - box2[2]/2, box2[1] - box2[3]/2,
                         box2[0] + box2[2]/2, box2[1] + box2[3]/2])
    return box_iou(box1.unsqueeze(0), box2.unsqueeze(0)).item()

def collate_fn(batch):
    images = torch.stack([item[0] for item in batch])
    labels = torch.stack([item[1] for item in batch])
    return images, labels

def train(model, train_dataloader, val_dataloader, optimizer, epochs=10, grid_size=7, num_classes=3):
    best_map = 0.0
    history = {'precision': [], 'recall': [], 'f1': [], 'map50': [], 'map5095': []}
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        total_objects = 0
        correct_boxes = 0
        
        for imgs, targets in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            preds = model(imgs)
            
            batch_size = preds.size(0)
            preds = preds.view(batch_size, grid_size * grid_size, -1)
            targets = targets.view(batch_size, grid_size * grid_size, -1)
            
            pred_obj = preds[..., 0]
            pred_box = preds[..., 1:5]
            pred_cls = preds[..., 5:5+num_classes]
            
            target_obj = targets[..., 0]
            target_box = targets[..., 1:5]
            target_cls = targets[..., 5:5+num_classes]
            
            obj_mask = target_obj == 1
            no_obj_mask = target_obj == 0
            
            obj_loss = nn.BCELoss()(pred_obj[obj_mask], target_obj[obj_mask])
            no_obj_loss = nn.BCELoss()(pred_obj[no_obj_mask], target_obj[no_obj_mask])
            coord_loss = nn.MSELoss()(pred_box[obj_mask], target_box[obj_mask])
            class_loss = nn.BCELoss()(pred_cls[obj_mask], target_cls[obj_mask])
            
            total_loss = 5*coord_loss + obj_loss + 0.5*no_obj_loss + class_loss
            total_loss.backward()
            optimizer.step()
            
            epoch_loss += total_loss.item()
            total_objects += obj_mask.sum().item()
            
            with torch.no_grad():
                for i in range(batch_size):
                    for j in range(grid_size * grid_size):
                        if obj_mask[i, j]:
                            pred_b = pred_box[i, j]
                            true_b = target_box[i, j]
                            if compute_iou(pred_b, true_b) > 0.5:
                                correct_boxes += 1
        
        model.eval()
        metrics = compute_map(model, val_dataloader, grid_size, num_classes)
                
        for key in history.keys():
            history[key].append(metrics[key])
        
        print(f"\nEpoch {epoch+1}")
        print(f"Precision: {metrics['precision']:.4f} | Recall: {metrics['recall']:.4f} | F1: {metrics['f1']:.4f}")
        print(f"mAP@0.5: {metrics['map50']:.4f} | mAP@0.5:0.95: {metrics['map5095']:.4f}")
        
        if metrics['map5095'] > best_map:
            best_map = metrics['map5095']
    
    return history

grid_size = 7
num_classes = 7
img_size = 416
batch_size = 3
lr = 0.001
epochs = 20


train_ds = YOLODataset(f"{dataset_dir}/train/images", f"{dataset_dir}/train/labels", img_size, grid_size)
val_ds = YOLODataset(f"{dataset_dir}/test/images", f"{dataset_dir}/test/labels", img_size, grid_size)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dl = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate_fn)

model = YOLOLike(grid_size=grid_size, num_classes=num_classes)
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=1e-4)

train(model, train_dl, val_dl, optimizer, epochs, grid_size, num_classes)

Epoch 1/3: 100%|██████████| 37/37 [00:15<00:00,  2.37it/s]



Epoch 1
Precision: 0.0174 | Recall: 0.0659 | F1: 0.0301
mAP@0.5: 0.0754 | mAP@0.5:0.95: 0.0199


Epoch 2/3: 100%|██████████| 37/37 [00:13<00:00,  2.68it/s]



Epoch 2
Precision: 0.0194 | Recall: 0.0700 | F1: 0.0296
mAP@0.5: 0.0604 | mAP@0.5:0.95: 0.0146


Epoch 3/3: 100%|██████████| 37/37 [00:14<00:00,  2.52it/s]



Epoch 3
Precision: 0.0291 | Recall: 0.1480 | F1: 0.0679
mAP@0.5: 0.1267 | mAP@0.5:0.95: 0.0358


{'precision': [0.029057333468362077,
  0.018756194880962813,
  0.04423483718704045,
  0.06287802032289834,
  0.045518764929859006,
  0.04626158492017438,
  0.048951048949426604,
  0.03641803674938292,
  0.03725273967753626,
  0.03865760407690159,
  0.09544573642990462,
  0.10762681064806465,
  0.11742245484073008,
  0.12444919785563853,
  0.1396195457037574,
  0.12353847547704438,
  0.1306165099209206,
  0.14873973377803795,
  0.12398624261712937,
  0.14142103629280664],
 'recall': [0.07593136789497285,
  0.06995923783581769,
  0.1479761114657336,
  0.19414162478015531,
  0.1679780073781422,
  0.1684519859542656,
  0.14001327138686004,
  0.11422883632574567,
  0.10247416815788471,
  0.1121433311108026,
  0.20542231489189283,
  0.19651151766077243,
  0.2594558725699634,
  0.27576073558860925,
  0.25812873255681784,
  0.25841311970249187,
  0.27253768127096994,
  0.24893354818002336,
  0.2768034884560808,
  0.21603943499705758],
 'f1': [0.03210644866607587,
  0.029581196141220182,
  0.06

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.models import resnet18
from torchvision.ops import box_iou
from scipy.optimize import linear_sum_assignment
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import numpy as np

device = torch.device("cpu")
CLASSES = ['elbow positive', 'fingers positive', 'forearm fracture', 'humerus fracture', 'humerus', 'shoulder fracture', 'wrist positive']
NUM_CLASSES = len(CLASSES)
IMG_SIZE = 256
MAX_DETECTIONS = 20

class YOLODetectionDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_dir = Path(img_dir)
        self.label_dir = Path(label_dir)
        self.transform = transform or T.Compose([
            T.Resize((IMG_SIZE, IMG_SIZE)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.images = sorted(self.img_dir.glob('*.jpg'))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label_path = self.label_dir / f"{img_path.stem}.txt"

        img = Image.open(img_path).convert("RGB")
        orig_w, orig_h = img.size
        img_tensor = self.transform(img)

        boxes = []
        labels = []
        if label_path.exists():
            with open(label_path) as f:
                for line in f:
                    cls, cx, cy, bw, bh = map(float, line.strip().split())
                    
                    scale_x = IMG_SIZE / orig_w
                    scale_y = IMG_SIZE / orig_h
                    
                    x1 = (cx - bw/2) * scale_x
                    y1 = (cy - bh/2) * scale_y
                    x2 = (cx + bw/2) * scale_x
                    y2 = (cy + bh/2) * scale_y
                    
                    boxes.append([x1, y1, x2, y2])
                    labels.append(int(cls))

        return {
            'image': img_tensor,
            'boxes': torch.tensor(boxes, dtype=torch.float32) if boxes else torch.zeros((0, 4)),
            'labels': torch.tensor(labels, dtype=torch.long) if labels else torch.zeros(0, dtype=torch.long)
        }

def collate_fn(batch):
    return {
        'image': torch.stack([x['image'] for x in batch]),
        'boxes': [x['boxes'] for x in batch],
        'labels': [x['labels'] for x in batch]
    }

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        layers = []
        for _ in range(num_layers-1):
            layers.extend([
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU()
            ])
            input_dim = hidden_dim
        layers.append(nn.Linear(hidden_dim, output_dim))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

class DETRLike(nn.Module):
    def __init__(self, num_classes, num_queries=MAX_DETECTIONS, hidden_dim=256):
        super().__init__()
        
        self.backbone = nn.Sequential(*list(resnet18(pretrained=False).children())[:-2])
        self.conv = nn.Conv2d(512, hidden_dim, 1)
        
        self.encoder_pos = nn.Parameter(torch.randn(1, hidden_dim, IMG_SIZE//32, IMG_SIZE//32))
        self.decoder_pos = nn.Parameter(torch.randn(num_queries, hidden_dim))
        
        self.transformer = nn.Transformer(
            d_model=hidden_dim,
            nhead=8,
            num_encoder_layers=3,
            num_decoder_layers=3,
            dim_feedforward=2048,
            dropout=0.1
        )
        
        self.class_embed = nn.Linear(hidden_dim, num_classes + 1)
        self.bbox_embed = MLP(hidden_dim, hidden_dim, 4, 3)

    def forward(self, x):
        features = self.backbone(x)
        src = self.conv(features)
        
        bs, c, h, w = src.shape
        src = src.flatten(2).permute(2, 0, 1)
        pos_enc = self.encoder_pos.expand(bs, -1, h, w).flatten(2).permute(2, 0, 1)
        src = src + pos_enc
        
        query_embed = self.decoder_pos.unsqueeze(1).repeat(1, bs, 1)
        
        hs = self.transformer(
            src=src,
            tgt=query_embed,
            src_key_padding_mask=None,
            tgt_key_padding_mask=None,
            memory_key_padding_mask=None
        )
        
        outputs_class = self.class_embed(hs)
        outputs_coord = self.bbox_embed(hs).sigmoid()
        
        return outputs_class.permute(1, 0, 2), outputs_coord.permute(1, 0, 2)

class HungarianMatcher(nn.Module):
    def __init__(self, cost_class=1, cost_bbox=5, cost_giou=2):
        super().__init__()
        self.cost_class = cost_class
        self.cost_bbox = cost_bbox
        self.cost_giou = cost_giou

    @torch.no_grad()
    def forward(self, outputs, targets):
        bs, num_queries = outputs["pred_logits"].shape[:2]
        
        indices = []
        for i in range(bs):
            out_prob = outputs["pred_logits"][i].softmax(-1)
            out_bbox = outputs["pred_boxes"][i]

            tgt_bbox = targets[i]["boxes"]
            tgt_ids = targets[i]["labels"]
            
            cost_class = -out_prob[:, tgt_ids]
            
            cost_bbox = torch.cdist(out_bbox, tgt_bbox, p=1)
            
            C = self.cost_bbox * cost_bbox + self.cost_class * cost_class
            C = C.reshape(num_queries, -1).cpu()
            
            indices.append(linear_sum_assignment(C))
        
        return [(torch.as_tensor(i, dtype=torch.int64), torch.as_tensor(j, dtype=torch.int64)) for i, j in indices]

def loss_fn(outputs, targets):
    pred_logits, pred_boxes = outputs
    
    matcher = HungarianMatcher()
    indices = matcher({"pred_logits": pred_logits, "pred_boxes": pred_boxes}, targets)
    
    src_logits = torch.cat([pred_logits[i][idx] for i, (idx, _) in enumerate(indices)])
    target_classes = torch.cat([t["labels"][j] for t, (_, j) in zip(targets, indices)])
    loss_cls = F.cross_entropy(src_logits, target_classes)
    
    src_boxes = torch.cat([pred_boxes[i][idx] for i, (idx, _) in enumerate(indices)])
    target_boxes = torch.cat([t["boxes"][j] for t, (_, j) in zip(targets, indices)])
    loss_bbox = F.l1_loss(src_boxes, target_boxes)
    
    return loss_cls + 5 * loss_bbox

def train(model, dataloader, optimizer, epochs=50):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
            images = batch['image'].to(device)
            targets = [
                {"boxes": b.to(device), "labels": l.to(device)}
                for b, l in zip(batch['boxes'], batch['labels'])
            ]
            
            outputs = model(images)
            loss = loss_fn(outputs, targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")

def evaluate(model, dataloader, conf_thresh=0.5):
    model.eval()
    results = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            images = batch['image'].to(device)
            outputs = model(images)
            pred_logits, pred_boxes = outputs
            
            probs = F.softmax(pred_logits, dim=-1)
            scores, labels = torch.max(probs, dim=-1)
            
            for i in range(images.size(0)):
                keep = scores[i] > conf_thresh
                pred = {
                    'boxes': pred_boxes[i][keep].cpu(),
                    'scores': scores[i][keep].cpu(),
                    'labels': labels[i][keep].cpu()
                }
                results.append((pred, batch['boxes'][i], batch['labels'][i]))
    
    aps = []
    tp_total = fp_total = fn_total = 0
    
    for iou_threshold in np.linspace(0.5, 0.95, 10):
        tp = fp = fn = 0
        
        for pred, gt_boxes, gt_labels in results:
            pred_boxes = pred['boxes']
            pred_labels = pred['labels']
            
            if len(pred_boxes) == 0:
                fn += len(gt_labels)
                continue
                
            ious = box_iou(pred_boxes, gt_boxes)
            
            matches = (ious > iou_threshold) & (pred_labels.unsqueeze(1) == gt_labels)
            matched_gt = set()
            matched_pred = set()
            
            for pred_idx, gt_idx in zip(*torch.where(matches)):
                if gt_idx not in matched_gt and pred_idx not in matched_pred:
                    matched_gt.add(gt_idx.item())
                    matched_pred.add(pred_idx.item())
            
            cur_tp = len(matched_gt)
            cur_fp = len(pred_boxes) - len(matched_pred)
            cur_fn = len(gt_labels) - len(matched_gt)
            
            if iou_threshold == 0.5:
                tp_total += cur_tp
                fp_total += cur_fp
                fn_total += cur_fn
            
            tp += cur_tp
            fp += cur_fp
            fn += cur_fn
        
        precision = tp / (tp + fp + 1e-6)
        recall = tp / (tp + fn + 1e-6)
        aps.append(precision)
    
    precision = tp_total / (tp_total + fp_total + 1e-6)
    recall = tp_total / (tp_total + fn_total + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)
    
    map50 = np.mean(aps[:1])
    map5095 = np.mean(aps)
    
    print(f"Precision@0.5: {precision:.4f}")
    print(f"Recall@0.5: {recall:.4f}")
    print(f"F1@0.5: {f1:.4f}")
    print(f"mAP@0.5: {map50:.4f}, mAP@0.5:0.95: {map5095:.4f}")

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")

model = DETRLike(NUM_CLASSES).to(device)
model.apply(init_weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

train_dataset = YOLODetectionDataset(f"{dataset_dir}/train/images", f"{dataset_dir}/train/labels")
val_dataset = YOLODetectionDataset(f"{dataset_dir}/test/images", f"{dataset_dir}/test/labels")

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=4, collate_fn=collate_fn
)

train(model, train_loader, optimizer, epochs=3)
evaluate(model, val_loader)

Epoch 1: 100%|██████████| 37/37 [00:20<00:00,  1.72it/s]


Epoch 1, Loss: 1.1518


Epoch 2: 100%|██████████| 37/37 [00:19<00:00,  1.91it/s]


Epoch 2, Loss: 0.7115


Epoch 3: 100%|██████████| 37/37 [00:20<00:00,  1.84it/s]


Epoch 3, Loss: 0.6239


Precision@0.5: 0.0272
Recall@0.5: 0.0784
F1@0.5: 0.0419
mAP@0.5: 0.0672, mAP@0.5:0.95: 0.0357


Как итог имеем, что свои имлементации работают не очень хорошо, лучше доверить эту работу врачам в таком случае.

Однако для модели уже составленной умными людьми получается сделать крутые результаты.